In [ ]:
!pip install omegaconf, einops, pytorch_lightning

In [1]:
from omegaconf import OmegaConf
from ldm.util import load_safetensors, get_input
import torch
from safetensors import safe_open
from einops import rearrange
import numpy as np

C:\Users\gfran\anaconda3\envs\trch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = load_safetensors(ckpt='../stable-diffusion-models/vae/diffusion_pytorch_model.safetensors')

# and then use it as model.encode(get_input(x), return_hidden=True)[1]

C:\Users\gfran\anaconda3\envs\trch\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [20]:
inp = torch.as_tensor(np.zeros((1, 256, 256, 3), np.float32)).cuda()

inp = get_input(inp)

inp.shape

torch.Size([1, 3, 256, 256])

In [21]:
_, h, hm = model.encode(inp, return_hidden=True)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 2.00 GiB total capacity; 1.51 GiB already allocated; 0 bytes free; 1.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
h.shape

torch.Size([1, 8, 18, 18])

In [5]:
print(model)

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (1): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
  

In [5]:
import numpy as np
import torch

In [1]:
from ldm.util import load_safetensors, get_input

C:\Users\gfran\anaconda3\envs\trch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = load_safetensors()

C:\Users\gfran\anaconda3\envs\trch\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [17]:
inp = torch.from_numpy(np.zeros((1, 64, 64, 3), np.float32)).cuda()

In [18]:
inp = get_input(inp)
latent, outputs = model.encode(inp, return_hidden=True)

In [21]:
torch.flatten(outputs[1])

tensor([-0.2858, -0.4730, -0.5461,  ..., -0.1655, -0.3258,  0.0375],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [ ]:
def get_image_features(inp, layer=-1): # needs inp with 3 channels
    inp = get_input(inp)
    gd, outputs = model.encode(inp, return_hidden=True)
    if layer > len(outputs):
        # return np.reshape(np.stack(outputs.hidden_states)[:, :, 0], (1, -1))
        result = torch.cat([torch.flatten(out) for out in outputs])
    if layer == -1:
        result = gd.sample()  # gd.mean()
    else:
        result = torch.flatten(outputs[layer])
    return np.asarray(result.cpu())

def get_features(inp, **args):
    return get_image_features(inp, **args)